In [135]:
print('yo')

In [136]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the trainidsng on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

log to Git

In [137]:
!git config --global user.email "basile.plus-gourdon@ens-paris-saclay.fr"
!git config --global user.name "basileplus"

log to HF

In [138]:
from huggingface_hub import notebook_login

notebook_login()

Load dataset

In [139]:
from datasets import load_dataset

raw_datasets = load_dataset("stanfordnlp/coqa")

Take a look a the dataset

In [140]:
print(raw_datasets)

In [141]:
print(raw_datasets['train'].info.features)

We format the dataset for it to fit the tutorial

In [142]:
def transform_example(batch):
    sources = []
    titles = []
    contexts = []
    questions = []
    answer_texts = []
    answer_starts = []
    answer_ends = []

    # Process each example in the batch
    for idx in range(len(batch["story"])):
        # num_answers = len(batch['answers'][idx]['input_text'])
        for num_question,question in enumerate(batch['questions'][idx]):
            # for a_idx in range(num_answers):
            sources.append(batch['source'][idx])
            titles.append(None)
            contexts.append(batch['story'][idx])
            questions.append(question)
            answer_texts.append(batch['answers'][idx]['input_text'][num_question])
            answer_starts.append(batch['answers'][idx]['answer_start'][num_question])
            answer_ends.append(batch['answers'][idx]['answer_end'][num_question])

    # Return a dictionary with the new structure
    return {
        'source': sources,
        'title': titles,
        'context': contexts,
        'question': questions,
        'answer_text': answer_texts,  # Changed to separate field
        'answer_start': answer_starts,  # Changed to separate field
        'answer_end': answer_ends
    }


In [143]:
from datasets import DatasetDict, concatenate_datasets

# Apply transformation and flatten the results
transformed_datasets = {}
for split, dataset in raw_datasets.items():
    # print(dataset.features)
    transformed_datasets[split] = dataset.map(transform_example, batched=True, remove_columns=dataset.column_names)
    # Flatten the list of lists into a single list of dicts

# Convert back to DatasetDict if necessary
transformed_dataset_dict = DatasetDict(transformed_datasets)



In [144]:
dataset = transformed_dataset_dict
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'title', 'context', 'question', 'answer_text', 'answer_start', 'answer_end'],
        num_rows: 108647
    })
    validation: Dataset({
        features: ['source', 'title', 'context', 'question', 'answer_text', 'answer_start', 'answer_end'],
        num_rows: 7983
    })
})

In [145]:
print(dataset["train"]["question"][0:25])
print(dataset["train"]["answer_text"][0:25])


Create tokenizer

In [146]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Always normalize data

In [147]:
questions = dataset["train"]["question"][0:3]
contexts = dataset["train"]["context"][0:3]
answers = dataset["train"]["answer_text"][0:3]

In [148]:
inputs = tokenizer(
    questions,
    contexts,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

- max_length to set the maximum length (here 100)
- truncation="only_second" to truncate the context (which is in the second position) when the question with its context is too long
- stride to set the number of overlapping tokens between two successive chunks (here 50)
- return_overflowing_tokens=True to let the tokenizer know we want the overflowing tokens

Context has been split in parts because too long

In [149]:
print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

with ``inputs['overflow_to_sample_mapping']`` we keep track of from wich subsection of the truncated context we are talking



We then correct the inputs to create correct labels :

*   If the answer is in the sequence, labels are start and end indices of answer
*   Otherwise start=end=0



In [150]:
print(answers)

offset mapping = beginning and end of token in real text

In [151]:
print(inputs["offset_mapping"])

overflow_to_sample_mapping indique après découpage du context (car trop long) à quel context appartient le ième élément de la liste

In [152]:
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

**feature** : input data which contains information

**ids** : id of the token

**.keys()** : allows to see every key of a dictionnary


In [153]:
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [154]:
start_positions = []
end_positions = []

questions = dataset["train"]["question"][0:3]
contexts = dataset["train"]["context"][0:3]
answers_text = dataset["train"]["answer_text"][0:3]
answers_start = dataset["train"]["answer_start"][0:3]
answers_end = dataset["train"]["answer_end"][0:3]

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer_text = answers_text[sample_idx]
    start_char = answers_start[sample_idx]
    end_char = answers_end[sample_idx]
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

print(start_positions)
print(end_positions)

Found indexes corresponds to tokens indexes

In [155]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Now let's wrapp it up in a function to apply it to the entire dataset

In [156]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = examples["question"]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []

    answers_text = examples["answer_text"]
    answers_start = examples["answer_start"]
    answers_end = examples["answer_end"]

    for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]
      start_char = answers_start[sample_idx]
      end_char = answers_end[sample_idx]
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label is (0, 0)
      if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [157]:
train_dataset = dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
len(dataset["train"]), len(train_dataset)

(108647, 151990)

## Preprocess validation data

We use the unused 'title' column to store from which example the generated feature comes

In [158]:
dataset['validation'] = dataset["validation"].rename_column('title','id')

To add an unique id to each example in order to map each feature to its originating example

In [159]:
def add_id(example,idx):
  example['id']=idx
  return example

In [160]:
dataset['validation'] = dataset['validation'].map(add_id, with_indices=True)

In [161]:
def preprocess_validation_examples(examples):
    questions = examples["question"]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [162]:
validation_dataset = dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns = dataset["validation"].column_names,
)
len(dataset["validation"]), len(validation_dataset)

Map:   0%|          | 0/7983 [00:00<?, ? examples/s]

(7983, 10951)

In [163]:
print(dataset['validation']['context'][0:4])
print(dataset['validation']['question'][0:4])
print(dataset['validation']['answer_text'][0:4])

# Postprocess

In [164]:
small_eval_set = dataset['validation'].select(range(10))
small_eval_set = small_eval_set.map(add_id, with_indices=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [165]:
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

trained_checkpoint = "WillHeld/bert-base-uncased-coqa"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Now that the preprocessing is done, we change the tokenizer back to the one we originally picked:

In [166]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [167]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [168]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [169]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [170]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue
                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )
                #print("answer ok : ", answers[-1], "start index :", start_index, " end index :", end_index)

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": str(example_id), "prediction_text": best_answer["text"]})

Modify the format of theoretical answer to match the metric attended format

In [171]:
import evaluate

metric = evaluate.load("squad")

theoretical_answers = []
for ex in small_eval_set:
    answer_dict = {"text": [ex["answer_text"]], "answer_start": [ex["answer_start"]]}
    theoretical_answers.append({"id": str(ex["id"]), "answers": answer_dict})


Modify references (=theoretical answers) to match the format attended by metric function

In [172]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 0.0, 'f1': 1.4814814814814814}

In [173]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)
    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []
        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    theoretical_answers = []
    for ex in tqdm(examples):
        answer_dict = {"text": [ex["answer_text"]], "answer_start": [ex["answer_start"]]}
        theoretical_answers.append({"id": str(ex["id"]), "answers": answer_dict})
    print("predict : ", predicted_answers)
    print("theoretical : ", theoretical_answers)
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [174]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'exact_match': 0.0, 'f1': 1.4814814814814814}

# Finetuning

In [175]:
model = AutoModelForQuestionAnswering.from_pretrained("basilePlus/bert-finetuned-squad")

In [176]:
from huggingface_hub import notebook_login

notebook_login()

In [177]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-coqa",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    # fp16=True, # if GPU can accelerate process
    push_to_hub=True,
)

To check if GPU is used

In [178]:
import torch
print(torch.cuda.is_available())  # Should return True if the GPU is available

# To see which GPU is being used (if any):
if torch.cuda.is_available():
  print(torch.cuda.current_device())
  print(torch.cuda.get_device_name(0))


In [179]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
# trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Don't forget to add the 'id' column in validation set

In [180]:
dataset['validation'] = dataset['validation'].map(add_id, with_indices=True)

In [181]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions


KeyboardInterrupt: 

In [ ]:
example_to_features_w = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features_w[feature["example_id"]].append(idx)

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(validation_dataset):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
compute_metrics(start_logits, end_logits, validation_dataset, dataset["validation"])

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

# Test the model

In [ ]:
from transformers import pipeline

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="basilePlus/bert-finetuned-coqa")

context = """
Transformers is a series of science fiction action films based on the Transformers franchise.[note 1] Michael Bay directed the first five live action films: Transformers (2007), Revenge of the Fallen (2009), Dark of the Moon (2011), Age of Extinction (2014), and The Last Knight (2017),[1][2][3] and has served as a producer for subsequent films. A sixth film Bumblebee, directed by Travis Knight, was released in 2018, while a seventh film, Rise of the Beasts, directed by Steven Caple Jr. was released in 2023.

The series has been distributed by Paramount Pictures, and DreamWorks Pictures worked on the first two films. The live action Transformers film series has received negative to mixed reception, with the exception of Bumblebee, which received positive reviews. It is the 13th-highest-grossing film series, with a total of $5.28 billion; the series films have generally been successful at the box office, with Dark of the Moon and Age of Extinction grossing over $1 billion each.
"""
question = "Who directed Bumblebee"
pipe(question=question, context=context)